<a href="https://colab.research.google.com/github/shaqas86/Langgraph_EcoSystem/blob/main/Agentic001_DLIMS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AGENTIC DRIVING LICENSE ISSUANCE MANGEMENT SYSTEM

PROJECT STACK = LANGGRAPH REACT AGENT

PROBLEM: USER WANTS ONLINE LEARNER SERVICE FOR DRIVING LICENSE

SOLUTION : APPLY ONLINE LEARNER WITH THE HELP OF DLIMS AGENTIC AI SYSTEM WITH EASE OF USE IN EVERY TYPE OF LANGUAGE

REQUIREMENT : ORGINAL CNIC [FRONT PIC 1 BACK PIC 2]

PASSPORT SIZE PICTURE [WITH WHITE BACKGROUND]

SCHEMA APPLICANT BIO DATA [NAME,FATHER NAME,DATE OF BIRTH, ADDRESS, BLOOD GROUP, HIGHT , CONTACT NO PERSONAL AND EMERGENCY NUMBER, CITY OR DISTRICT ]

SCHEMA FOR DRIVING LICENSE CATEGORY [MCYCLE, MCAR, LTV , TR. AGRI, TR. COMM, INVALID CARRIAGE]

POLICES:

## SERVICES

<LI>  ONLINE LEARNER